In [ ]:
import teradatasql
import pandas as pd


# you can connect directly to GDW table downloaded or use csv if you are not connected to the network

# download centre mapping from GDW
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    lg_hier = pd.read_sql('select * from dwpviewa.LG_Hier_Centre where date between from_date and to_date and lg_centre_l14_key in (\'RB4892\',\'RB4152\',\'RB7062\')', connect)

# download product mapping from GDW
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    lg_prod = pd.read_sql('select * from dwpviewa.LG_Hier_Product where date between from_date and to_date', connect)
    
# download the latest credit risk file and dont forget to change the month
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    df = pd.read_sql('select * from finiq.CRDBEXTRACT where month_key in (\'202009\')', connect)


# create mapping table to csv
lg_hier.to_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Map_Centre.csv')
lg_prod.to_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Map_Product.csv')



In [23]:
import teradatasql
import pandas as pd

start = time.time()

mysql = """
select * from 
dwpviewa.LG_Hier_Centre 
where date between from_date and to_date 
and lg_centre_l14_key in (\'RB4892\',\'RB4152\',\'RB7062\')  """


# you can connect directly to GDW table downloaded or use csv if you are not connected to the network

# download centre mapping from GDW
with teradatasql.connect(host='tdp5dbcw.teradata.westpac.com.au', user='M118954', password='mygdw2021!') as connect:
    test = pd.read_sql(mysql, connect)

    
    
finish = time.time()

print (finish - start)

32.98898124694824


In [10]:
def leadZero(a):
    mystr = str(a)
    if len(mystr) == 5:
        mystr = '0' + mystr
    if len(mystr) == 4:
        mystr = '00' + mystr
    if len(mystr) == 3:
        mystr = '000' + mystr
    else:
        mystr = mystr
    return mystr

# ECC Only
def CR_ECC(Class,ECC,RWA):
    if Class in ['SMECORP','SLIPRE','CORP']:
        calc = ECC
    else:
        calc = 0
    return calc

# RWA Only
def CR_RWA(Class,ECC,RWA):
    if Class in ['SMECORP','SLIPRE','CORP']:
        calc = 0
    else:
        calc = RWA * 0.0875
    return calc

# Total Credit Risk (ECC + RWA)
def CR_Tot(Class,ECC,RWA):
    if Class in ['SMECORP','SLIPRE','CORP']:
        calc = ECC
    else:
        calc = RWA * 0.0875
    return calc

In [6]:
import pandas as pd
import time


start = time.time()
# download the file from spreadsheet
risk = pd.read_excel(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\RECSOURCEDATA.xlsx', sheet_name='RISK DATA') 
coi = pd.read_excel(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\RECSOURCEDATA.xlsx', sheet_name='COI DATA') 

# fill all the NAs
risk['CENTRE_ID'] = risk['CENTRE_ID'].fillna(0)
risk['PRODUCT_CODE'] = risk['PRODUCT_CODE'].fillna(0)

# convert data type
risk = risk.astype({'CENTRE_ID': 'int64','PRODUCT_CODE':'int64'})

finish = time.time()

print ('It took around ' + str((finish-start)/60) + ' minutes')



It took around 3.3130995512008665 minutes


In [13]:
a = risk[['CENTRE_ID','ASSET_SUBCLASS','DIVISION','BUSINESS_UNIT','PRODUCT_CODE','PRODUCT_1','ECC_AMT','RWA']]
b = coi[['CENTRE_ID','ASSET_SUBCLASS','l13','PRODUCT_CODE','prod10','ECC','RWA']]

finalrec = pd.merge(a,b,on=['CENTRE_ID','PRODUCT_CODE','ASSET_SUBCLASS'],how='left')
finalrec = finalrec[['CENTRE_ID', 'PRODUCT_CODE','ASSET_SUBCLASS','DIVISION', 'BUSINESS_UNIT', 'l13', 'PRODUCT_1', 'prod10','ECC_AMT', 'RWA_x',  'ECC', 'RWA_y']]

finalrec.to_csv((r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\FinalRec.csv'))

In [11]:
import time
import pandas as pd

start = time.time()
lg_hier = pd.read_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Map_Centre.csv')
lg_hier['LG_Centre_Id'] = lg_hier['LG_Centre_Id'].str.strip()    # remove trailing spaces


aggMap = pd.read_excel(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\Check Credit Risk.xlsm', sheet_name='Ctr Mapping') 
#aggMap = aggMap.rename(columns={'CENTRE_ID':'LG_Centre_Id'})

# convert to string
aggMap = aggMap.astype({'Centre': 'str'})
aggMap = aggMap.astype({'Mapping': 'str'})

# rename column
aggMap = aggMap.rename(columns={'Centre':'LG_Centre_Id'})
aggMap = aggMap[['LG_Centre_Id','Mapping']]

# remove trailing spaces
aggMap['LG_Centre_Id'] = aggMap['LG_Centre_Id'].str.strip()    # remove trailing spaces
aggMap['Mapping'] = aggMap['Mapping'].str.strip()    # remove trailing spaces
aggMap['LG_Centre_Id'] = aggMap.apply(lambda x:leadZero(x['LG_Centre_Id']),axis=1)    # apply function to fix leading zero


# run merging sql
aggMap = pd.merge(aggMap,lg_hier,how="left",on=['LG_Centre_Id'])
aggMap = aggMap[['LG_Centre_Id','Mapping','LG_Centre_L14_Name','LG_Centre_L13_Name','LG_Centre_L12_Name']]


#rename columns
aggMap = aggMap.rename(columns={'LG_Centre_L14_Name':'L14',
                                'LG_Centre_L13_Name':'L13',
                                'LG_Centre_L12_Name':'L12',
                                'LG_Centre_Id':'Centre_Id',
                                'Mapping':'LG_Centre_Id'
                               })

# run second mergin sql
aggMap = pd.merge(aggMap,lg_hier,how="left",on=['LG_Centre_Id'])
aggMap = aggMap[['Centre_Id','LG_Centre_Id','L14','L13','L12','LG_Centre_L14_Name','LG_Centre_L13_Name','LG_Centre_L12_Name']]

#rename columns 2
aggMap = aggMap.rename(columns={'LG_Centre_L14_Name':'L14_Agg',
                                'LG_Centre_L13_Name':'L13_Agg',
                                'LG_Centre_L12_Name':'L12_Agg',
                                'LG_Centre_Id':'Agg_Id'
                               })



finish = time.time()
duration = round(((finish-start)/60),2)


if (finish-start)<60:
    duration = round((finish-start),2)
    print ('it took around ' + str(duration) + ' seconds')
else:
    duration = round(((finish-start)/60),2)
    print ('it took around ' + str(duration) + ' minutes')

    

it took around 43.57 seconds


In [6]:
aggMap.to_csv(r'C:\Users\m118954\OneDrive - Westpac Group\LOB Project\OldData.csv')